In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D

# Carregar dados do arquivo CSV
file_path = "caminho do arquivo"  # Certifique-se de ajustar o caminho conforme necessário
df = pd.read_csv(file_path, delimiter=';', encoding='utf-8')

# Escolher as colunas relevantes
df = df[['response', 'code0']]

# Dividir os dados em conjuntos de treinamento e teste
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Tokenizar textos usando o tokenizer do RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_tokens = tokenizer(list(train_data['response']), padding=True, truncation=True, return_tensors='tf')
test_tokens = tokenizer(list(test_data['response']), padding=True, truncation=True, return_tensors='tf')

# Carregar o modelo pré-treinado do RoBERTa
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

# Criar o modelo de classificação com camadas adicionais usando a API funcional do Keras
inputs = Input(shape=(None,), dtype=tf.int32)
roberta_outputs = roberta_model(inputs)['pooler_output']
output = Dense(1, activation='sigmoid')(roberta_outputs)

model = tf.keras.Model(inputs=inputs, outputs=output)

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
labels = np.array(train_data['code0'])
model.fit(train_tokens['input_ids'], labels, epochs=3, batch_size=32, validation_split=0.2)

# Avaliar o modelo no conjunto de teste
test_labels = np.array(test_data['code0'])
loss, accuracy = model.evaluate(test_tokens['input_ids'], test_labels)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/3
